In [49]:
def get_infos_curitiba():

    import pandas as pd
    import requests
    import bs4

    ## Request site
    response = requests.get('https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Curitiba')

    ## beautiful soup object
    soup = bs4.BeautifulSoup(response.content, 'html.parser')

    # tabelas da wikipedia
    infos_tabela = soup.findAll('table', {'class','wikitable'})

    bairros_info_list = []

    ## para cada tabela
    for tabela in infos_tabela:

        # colunas
        columns = [i.text.replace('\n','') for i in tabela.findAll('th')]

        # valores da tabela
        table_values = [i.text.replace('\xa0','').replace('\n','') for i in tabela.findAll('td')][1::]
        table_values_list = [] 
        for i in range(0,len(table_values),7): 
            table_values_list.append(table_values[i:i+7])

        # preenchenco dicionário
        for b in table_values_list:

            bairros_info_dict = {}

            bairros_info_dict['Bairro'] = b[0]
            bairros_info_dict['Área (km²)'] = b[1]
            bairros_info_dict['Homens'] = b[2]
            bairros_info_dict['Mulheres'] = b[3]
            bairros_info_dict['Total'] = b[4]
            bairros_info_dict['Domicilios particulares'] = b[5]
            bairros_info_dict['Rendimento mensal médio porresponsáveis dos domicílios (R$)'] = b[6]

            bairros_info_list.append(bairros_info_dict)
    
    return pd.DataFrame(bairros_info_list)


In [50]:
get_infos_curitiba()

,Bairro,Área (km²),Homens,Mulheres,Total,Domicilios particulares,Rendimento mensal médio porresponsáveis dos domicílios (R$)
0,Ganchinho,"11,20",3667,3658,7325,1921,"767,35"
1,Sitio Cercado,"11,12",50631,51779,102410,27914,"934,95"
2,Umbará,"22,47",7280,7315,14595,17064,"908,70"
3,Abranches,"4,32",5463,5702,11165,3154,"1009,67"
4,Atuba,"4,27",6156,6476,12632,3627,"1211,60"
...,...,...,...,...,...,...,...
71,Santo Inácio,"2,72",2940,3097,6037,1683,"1518,26"
72,São Braz,"5,01",11163,11956,23119,6600,"1206,50"
73,São João,"3,03",1441,1509,2950,814,"1166,03"
74,Seminário,"2,13",3350,4045,7395,2243,"3210,65"


In [58]:
import os
from google.cloud import storage
import pandas as pd
import numpy as np
from flask import Flask
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import datetime
import re
from unidecode import unidecode

link_anuncio = 'https://www.apolar.com.br/alugar/curitiba/bigorrilho/alugar-residencial-apartamento-curitiba-bigorrilho-105921'

driver = webdriver.Chrome()
driver.get(link_anuncio)
# print(link_anuncio)
# time.sleep(2)
page = bs4.BeautifulSoup(driver.page_source, 'html.parser')

In [82]:
[i.text.replace('A partir de','Aluguel').replace('   ','').replace('\n','') .replace('Outros valores','').replace('/mês','').replace('  ',', ').strip() for i in page.findAll('div',{'class':'price-box'})]

['Aluguel R$ 4.000,00, , Condomínio R$ 1.129,00, Seguro Incêndio R$ 48,51, Total R$ 5.177,51']

In [65]:
anuncio_infos = {}

anuncio_infos['site'] = 'Apolar'
anuncio_infos['data_coleta'] = datetime.datetime.today().strftime('%Y-%m-%d')
try:
    anuncio_infos['titulo'] = page.findAll('h1', {'class':'property-title'})[0].text
except:
    anuncio_infos['titulo'] = np.nan
try:
    anuncio_infos['link'] = link_anuncio
except:
    anuncio_infos['link'] = np.nan
try:
    anuncio_infos['endereco'] = page.findAll('a', {'class':'property-address'})[0].text
except:
    anuncio_infos['endereco'] = np.nan
try:
    anuncio_infos['valores'] = page.findAll('table', {'class':'sohtectable-style'})[0].text
except:
    anuncio_infos['valores'] = np.nan
try:
    anuncio_infos['atributos'] = ', '.join([i.text for i in page.findAll('li',{'class':'highlight'})]).replace('\n','').replace('                        ',' ').replace('                      ', ' ').replace('(','').replace(')','').strip().replace('  ',', ')
except:
    anuncio_infos['atributos'] = np.nan
try:
    anuncio_infos['descricao'] = page.findAll('div', {'class':'description'})[0].text
except:
    anuncio_infos['descricao'] = np.nan

In [66]:
anuncio_infos


{'site': 'Apolar',
 'data_coleta': '2024-02-27',
 'titulo': '\n                Apartamento Residencial para Locação no Bigorrilho\n                ref. 105921',
 'link': 'https://www.apolar.com.br/alugar/curitiba/bigorrilho/alugar-residencial-apartamento-curitiba-bigorrilho-105921',
 'endereco': '\n                Rua Martim Afonso, 1918, Bigorrilho - Curitiba\n              ',
 'valores': nan,
 'atributos': '146 m², 2 banheiros, 1 vaga, 3 quartos, 1 suite',
 'descricao': '\n                          Apartamento com\n                          3 quartos\n                          no Bairro Bigorrilho\n                      Imóvel com gestão exclusiva ApolarMorar em um excelente apartamento em uma região nobre de Curitiba, Rua Martin Afonso é uma experiência que oferece uma qualidade de vida excepcional. O Bairro Bigorrilho é conhecido por sua alta qualidade de vida, planejamento urbano eficiente e beleza natural. Viver em uma região nobre da cidade acrescenta ainda mais valor à experiên